In [54]:
import re
import csv
from nltk.corpus import wordnet as wn

In [107]:
adj_exceptions = ['gratinated', 'indian', 'greek']

def is_adjective(expr):
    global adj_exceptions
    for word in expr.split():
        s = wn.synsets(word)        
        if (len(s) == 0 or s[0].pos() == 'n') and word not in adj_exceptions:
            return False
    return True


def correct(s):
    return s.replace(
        ',', " and").replace(
        '-', ' ').replace(        
        '&', "and").replace(
        "garnish", '').replace(
        "bell pepper", "bellpepper").replace(
        "or ", '')

def parse_name(s):
    parsed = re.split('/|with|"|and', correct(s))
    parsed = [x.strip().lower() for x in parsed if len(x.strip())]
    items = []
    acc = ""
    for i, w in enumerate(parsed):
        if i < len(parsed) - 1 and is_adjective(w):
                acc += w + ' '
        else:
            items.append(acc + w)
            acc = ''
    return items

In [108]:
with open("./sv_restaurant_data.csv", 'r') as f:
    r = csv.DictReader(f)
    for line in r:
        print(parse_name(line["name"]))

['beef broth']
['cream of corn soup']
['pasta', 'ham', 'small salad']
['falafel balls', 'tomato eggplant salad', 'mint yogurt']
['grilled hake', 'ratatouille', 'olive potatoes']
['poppy seed cheese cake']
['beef broth']
['creamed garlic soup']
['pumpkin spinach pan', 'dates', 'nuts', 'bulgur']
['käsekrainer', 'french fries', 'onion mustard']
['gratinated chicken filet', 'tomato', 'mozzarella', 'bell peper', 'zucchini', 'herb pasta']
['latte macchiato cake']
['beef broth']
['minestrone']
['potato vegetable strudel', 'leaf salad', 'herb dip']
['breaded fried chicken schnitzel', 'rice']
['wok noodles', 'turkey strips', 'vegetables', 'cashews']
['yogurt fruit cake']
['beef broth']
['cream of cauliflower soup']
['grilled leberkäse', 'green beans', 'dill', 'potatoes']
['jambalaya', 'bulgur']
['esterházy beef escalope', 'green pasta', 'root vegetables']
['apricot cake']
['beef broth']
['creamed basil soup']
['buttermilk', 'vegetable curry', 'spelt']
['breaded fried plaice', 'parsley potatoes'